In [1]:
import datetime
BU = 'BEAUTY'
today = datetime.date.today()
carpeta_input = R'S:\BI\16. DATA SCIENCE\9_Buffers\BuffersBasicosV1\input'
carpeta_output = R'S:\BI\16. DATA SCIENCE\9_Buffers\BuffersBasicosV1\output'

In [2]:

import get_data as gd


div =   {'BEAUTY': "[Division Code] = 'BEAUTY'", 
            'JEW': "[Division Code] IN ('W-JEW', 'M-JEW')",
            'ACC': "[Division Code] IN ('W-ACC', 'M-ACC')",
        'CALZADO':"[Division Code] in ('W-SHO','M-SHO')",
           'ROPA':"[Division Code] in ('W-CLO','M-CLO')"} 
   
end_date = str(today)

# vale, la proyeccion es con base en el año anterior entonces, se necesita un query grande, en este caso se toma desde la primera semana del año anterior                                    
df_inv_ventas_hist = gd.get_inv_venta_hist(BU, div, '2022-01-09', end_date, carpeta_input, carpeta_output, 33)

Consultando inventarios... 
Consultando ventas... 
Consultando clones... 
Preparando data... 
Identificando características de tiendas y productos....


In [14]:
import pandas as pd
import numpy as np
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df_inv_ventas_hist['ID'].str[6:].value_counts().min()

28

In [5]:
#T117 2112513083
print("Detectando semanas faltantes... ") #-------------------------------------------------------------------------------------------------
# Aquí se crea un rango de fechas para cada SKU y tienda

# Creando un dataframe con los datos faltantes
datos_faltantes = pd.DataFrame(columns=df_inv_ventas_hist.columns)
numero_datos_faltantes = pd.DataFrame(columns=['Location Code', 'SKU', 'Cantidad'])

SKUS = df_inv_ventas_hist['SKU'].unique()
for sku in SKUS: 
    df = df_inv_ventas_hist[df_inv_ventas_hist['SKU'] == sku] # for
    tiendas = df['Location Code'].unique()
    for tienda in tiendas:
        df_tienda = df[df['Location Code'] == tienda]

        first_week = df_tienda["Date"].min().strftime("%Y-%m-%d")
        last_week = df_tienda["Date"].max().strftime("%Y-%m-%d")

        # Se crea un rango de fechas con el fin de identificar si hay datos faltantes
        weeks = pd.date_range(start=first_week, end=last_week, freq='W')

        # Se crea un dataframe con el rango de fechas
        df_weeks = pd.DataFrame(weeks, columns=['Date'])

        df_tienda = pd.merge(df_weeks, df_tienda, how='left', on='Date')

        # Se rellenan los datos faltantes con nan
        df_tienda = df_tienda.fillna(np.nan)

        # Se rellenan los vacios con interpolacion lineal * deje ID con nans para identificarlos
        
        df_tienda['Inventario'] = df_tienda['Inventario'].interpolate()
        df_tienda['Ventas'] = df_tienda['Ventas'].interpolate()

        df_tienda['Location Code'].fillna(tienda, inplace=True)
        df_tienda['SKU'].fillna(sku, inplace=True)

        Cantidad = df_tienda['ID'].isna().sum()
        faltantes = pd.DataFrame({'Location Code': tienda, 'SKU': sku, 'Cantidad': Cantidad}, index=[0])
        numero_datos_faltantes = pd.concat([numero_datos_faltantes, faltantes])

        # Se agrega en el dataframe de datos faltantes
        datos_faltantes = pd.concat([datos_faltantes, df_tienda[df_tienda['ID'].isna()]])

        # tiempo interpolacion 1 min 40 seg
        # 54 seg
        # 48 seg

df_inv_ventas_hist['Date'] = pd.to_datetime(df_inv_ventas_hist['Date'])
df_inv_ventas_hist = pd.concat([df_inv_ventas_hist, datos_faltantes]).reset_index(drop=True)
df_inv_ventas_hist = df_inv_ventas_hist.sort_values(by=['SKU', 'Location Code', 'Date']).reset_index(drop=True) ##

print('Porcentaje de datos interpolados:', str((df_inv_ventas_hist['ID'].isnull().sum() / len(df_inv_ventas_hist))*100) + '%')

Detectando semanas faltantes... 
Porcentaje de datos interpolados: 1.2445220300765552%


In [12]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    
    dataset = tf.data.Dataset.from_tensor_slices(series)

    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    # Single batch
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))

    dataset = dataset.map(lambda window: (window[:-1], window[-1]))

    dataset = dataset.shuffle(shuffle_buffer)
    
    dataset = dataset.batch(batch_size).prefetch(1)
    
    return dataset

In [22]:
from datetime import datetime



In [34]:


def proyeccion_RNN(producto, tienda):
    print(producto, tienda)
        
    df = df_inv_ventas_hist[(df_inv_ventas_hist['SKU'] == producto) & (df_inv_ventas_hist['Location Code'] == tienda)].reset_index(drop=True)
    df = df[['Date', 'Ventas']]
    df['Ventas']= df['Ventas'].astype(int)

    # ordenando por fecha
    df = df.sort_values(by=['Date']).reset_index(drop=True)
    df['time'] = range(len(df))

    time = df['time'].values
    series = df['Ventas'].values

    # Define the split time
    split_time = int(round(len(df)*0.80,0))

    # Get the train set 
    time_train = time[:split_time]
    x_train = series[:split_time]

    # Get the validation set
    time_valid = time[split_time:]
    x_valid = series[split_time:]    

    window_size = 4
    batch_size = 30
    shuffle_buffer_size = 1000

    dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)    

    l0 = tf.keras.layers.Dense(1, input_shape=[window_size])

    model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=64, kernel_size=3,
                        strides=1, padding="causal",
                        activation="relu",
                        input_shape=[window_size, 1]),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x: x * 400)
    ])

    #model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(learning_rate=1e-6, momentum=0.9), metrics=["accuracy"])
    model.compile(loss=tf.keras.losses.Huber(), optimizer=tf.keras.optimizers.SGD(learning_rate=1e-6, momentum=0.9), metrics=["mae"])

    history = model.fit(dataset,epochs=100)

    '''forecast = []
    for time in range(len(series) - window_size):
        forecast.append(model.predict(series[time:time + window_size][np.newaxis]))

    forecast = forecast[split_time - window_size:]
    results = np.array(forecast).squeeze()

    #Convert to a numpy array and drop single dimensional axes
    results = np.array(forecast).squeeze()
    results = np.round(abs(results), 0)

    # Overlay the results with the validation set
    time = df['time'].values
    series = df['Ventas'].values
    forecast = forecast[split_time - window_size:]
    # plot_series(time_valid, (x_valid, abs(results)))

    #print("Real: ", sum(x_valid[-7:]))
    #print("Proyectada: ", sum(abs(results[-7:])))

    #print(tf.keras.metrics.mean_squared_error(x_valid, results).numpy())
    #print(tf.keras.metrics.mean_absolute_error(x_valid, results).numpy())'''

    ultima_fecha = str(df.loc[(len(df)-1),'Date'])[:10]
    ultima_fecha = datetime.strptime(ultima_fecha, '%Y-%m-%d')
    from datetime import timedelta

    size_proyeccion = 8
    fechas_proyeccion = []
    forecast = []

    #series = series[:split_time]

    for i in range(size_proyeccion):
        forecast.append(model.predict(series[-window_size:][np.newaxis]))
        series = np.append(series, abs(int(forecast[-1].squeeze())))
        ultima_fecha = ultima_fecha + timedelta(days=7)
        fechas_proyeccion.append(ultima_fecha)
        i = i + 1
        
    #print('Real: ', sum(series2[split_time:split_time+8]))
    #print('Proyectada: ', sum(series[-8:]))
    #print("Actual: ", (series2[split_time:split_time+8]).mean()*8)
    #plot_series(time_valid[:8], (series2[split_time:split_time+8], series[-8:]))
    proyeccion = series[-size_proyeccion:]
    
    b = pd.DataFrame({'Fecha':fechas_proyeccion, 'Proyeccion':proyeccion})
    b['SKU'] = producto
    b['Tienda'] = tienda
    #Proyecciones = pd.concat([Proyecciones, b], axis=0)


    mae=history.history['mae'][-1]
    loss=history.history['loss'][-1]

    tf.keras.backend.clear_session()

    return b, loss, mae

#producto = '2112513083'
#tienda = 'T117'    
#bd, loss, mae = proyeccion_RNN(producto, tienda)

2112513083 T117
Epoch 1/100
2/2 [==============================] - 3s 29ms/step - loss: 24.2494 - mae: 24.7494
Epoch 2/100
2/2 [==============================] - 0s 5ms/step - loss: 23.0572 - mae: 23.5572
Epoch 3/100
2/2 [==============================] - 0s 4ms/step - loss: 20.5634 - mae: 21.0634
Epoch 4/100
2/2 [==============================] - 0s 6ms/step - loss: 17.0692 - mae: 17.5667
Epoch 5/100
2/2 [==============================] - 0s 5ms/step - loss: 13.1285 - mae: 13.6257
Epoch 6/100
2/2 [==============================] - 0s 4ms/step - loss: 9.4931 - mae: 9.9925
Epoch 7/100
2/2 [==============================] - 0s 7ms/step - loss: 7.4823 - mae: 7.9714
Epoch 8/100
2/2 [==============================] - 0s 9ms/step - loss: 6.7257 - mae: 7.2030
Epoch 9/100
2/2 [==============================] - 0s 9ms/step - loss: 6.9197 - mae: 7.3982
Epoch 10/100
2/2 [==============================] - 0s 6ms/step - loss: 7.3284 - mae: 7.8143
Epoch 11/100
2/2 [==============================] - 

In [38]:
Proyecciones = pd.DataFrame(columns=['SKU', 'Tienda', 'Fecha', 'Proyeccion'])
Status = pd.DataFrame(columns=['SKU', 'Tienda', 'loss', 'mae'])

for producto in df_inv_ventas_hist['SKU'].unique():
    df = df_inv_ventas_hist[df_inv_ventas_hist['SKU'] == producto]
    for tienda in df['Location Code'].unique():
        bd, loss, mae = proyeccion_RNN(producto, tienda)
        Proyecciones = pd.concat([Proyecciones, bd], axis=0)
        Status = Status.append({'SKU':producto, 'Tienda':tienda, 'loss':loss, 'mae':mae}, ignore_index=True)

1215560051 E002
Epoch 1/100
3/3 [==============================] - 3s 24ms/step - loss: 5.8655 - mae: 6.3130
Epoch 2/100
3/3 [==============================] - 0s 10ms/step - loss: 4.9336 - mae: 5.4073
Epoch 3/100
3/3 [==============================] - 0s 9ms/step - loss: 4.1087 - mae: 4.5890
Epoch 4/100
3/3 [==============================] - 0s 8ms/step - loss: 3.9230 - mae: 4.3964
Epoch 5/100
3/3 [==============================] - 0s 8ms/step - loss: 4.0825 - mae: 4.5623
Epoch 6/100
3/3 [==============================] - 0s 8ms/step - loss: 3.9387 - mae: 4.4187
Epoch 7/100
3/3 [==============================] - 0s 11ms/step - loss: 3.5560 - mae: 4.0292
Epoch 8/100
3/3 [==============================] - 0s 9ms/step - loss: 3.2453 - mae: 3.7112
Epoch 9/100
3/3 [==============================] - 0s 10ms/step - loss: 3.1560 - mae: 3.6141
Epoch 10/100
3/3 [==============================] - 0s 9ms/step - loss: 3.0686 - mae: 3.5226
Epoch 11/100
3/3 [==============================] - 0s 10ms

C:\Users\fcolin\AppData\Local\Temp\ipykernel_42424\1929561199.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Status = Status.append({'SKU':producto, 'Tienda':tienda, 'loss':loss, 'mae':mae}, ignore_index=True)


Epoch 1/100
3/3 [==============================] - 7s 41ms/step - loss: 3.3808 - mae: 3.8611
Epoch 2/100
3/3 [==============================] - 0s 19ms/step - loss: 3.0073 - mae: 3.4731
Epoch 3/100
3/3 [==============================] - 0s 16ms/step - loss: 2.8154 - mae: 3.2967
Epoch 4/100
3/3 [==============================] - 0s 16ms/step - loss: 2.8151 - mae: 3.2793
Epoch 5/100
3/3 [==============================] - 0s 18ms/step - loss: 2.6824 - mae: 3.1429
Epoch 6/100
3/3 [==============================] - 0s 17ms/step - loss: 2.4689 - mae: 2.9391
Epoch 7/100
3/3 [==============================] - 0s 17ms/step - loss: 2.4484 - mae: 2.8944
Epoch 8/100
3/3 [==============================] - 0s 18ms/step - loss: 2.3784 - mae: 2.8211
Epoch 9/100
3/3 [==============================] - 0s 12ms/step - loss: 2.2209 - mae: 2.6540
Epoch 10/100
3/3 [==============================] - 0s 13ms/step - loss: 2.1669 - mae: 2.6038
Epoch 11/100
3/3 [==============================] - 0s 14ms/step - lo

C:\Users\fcolin\AppData\Local\Temp\ipykernel_42424\1929561199.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Status = Status.append({'SKU':producto, 'Tienda':tienda, 'loss':loss, 'mae':mae}, ignore_index=True)


Epoch 1/100
3/3 [==============================] - 7s 29ms/step - loss: 3.6589 - mae: 4.1383
Epoch 2/100
3/3 [==============================] - 0s 14ms/step - loss: 2.7581 - mae: 3.2286
Epoch 3/100
3/3 [==============================] - 0s 10ms/step - loss: 2.3387 - mae: 2.8196
Epoch 4/100
3/3 [==============================] - 0s 11ms/step - loss: 2.6568 - mae: 3.1301
Epoch 5/100
3/3 [==============================] - 0s 10ms/step - loss: 2.6913 - mae: 3.1565
Epoch 6/100
3/3 [==============================] - 0s 12ms/step - loss: 2.1942 - mae: 2.6532
Epoch 7/100
3/3 [==============================] - 0s 11ms/step - loss: 1.9965 - mae: 2.4481
Epoch 8/100
3/3 [==============================] - 0s 10ms/step - loss: 2.0215 - mae: 2.4776
Epoch 9/100
3/3 [==============================] - 0s 16ms/step - loss: 1.8149 - mae: 2.2685
Epoch 10/100
3/3 [==============================] - 0s 16ms/step - loss: 1.5182 - mae: 1.9619
Epoch 11/100
3/3 [==============================] - 0s 17ms/step - lo

...

C:\Users\fcolin\AppData\Local\Temp\ipykernel_42424\1929561199.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Status = Status.append({'SKU':producto, 'Tienda':tienda, 'loss':loss, 'mae':mae}, ignore_index=True)


Epoch 1/100
3/3 [==============================] - 5s 20ms/step - loss: 19.0661 - mae: 19.5544
Epoch 2/100
3/3 [==============================] - 0s 12ms/step - loss: 15.8295 - mae: 16.3206
Epoch 3/100
3/3 [==============================] - 0s 10ms/step - loss: 11.0533 - mae: 11.5500
Epoch 4/100
3/3 [==============================] - 0s 7ms/step - loss: 7.7041 - mae: 8.1925
Epoch 5/100
3/3 [==============================] - 0s 8ms/step - loss: 7.4568 - mae: 7.9493
Epoch 6/100
3/3 [==============================] - 0s 8ms/step - loss: 8.4653 - mae: 8.9568
Epoch 7/100
3/3 [==============================] - 0s 7ms/step - loss: 8.5087 - mae: 9.0034
Epoch 8/100
3/3 [==============================] - 0s 7ms/step - loss: 6.7415 - mae: 7.2185
Epoch 9/100
3/3 [==============================] - 0s 12ms/step - loss: 5.1384 - mae: 5.6349
Epoch 10/100
3/3 [==============================] - 0s 8ms/step - loss: 4.5760 - mae: 5.0734
Epoch 11/100
3/3 [==============================] - 0s 10ms/step - lo

C:\Users\fcolin\AppData\Local\Temp\ipykernel_42424\1929561199.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Status = Status.append({'SKU':producto, 'Tienda':tienda, 'loss':loss, 'mae':mae}, ignore_index=True)


Epoch 1/100


KeyboardInterrupt: 

In [39]:
Status.to_csv('Status.csv', index=False)
Proyecciones.to_csv('Proyecciones.csv', index=False)

,SKU,Tienda,loss,mae
0,1215560051,E002,0.749583,1.160705
1,1215560051,T001,1.130049,1.572009
2,1215560051,T003,0.629425,1.061737
3,1215560051,T004,0.712766,1.082541
4,1215560051,T005,0.898344,1.287563
